In [ ]:
pip install tensorflow opencv-python

In [ ]:
import numpy as np
import pandas as pd 
import os
import glob
from sklearn.model_selection import train_test_split
import shutil
import zipfile
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3, VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Divide the folders

In [ ]:
def split(data_path,train,validation,split_size=0.1):
    folders = os.listdir(data_path)
    for f in folders:
        fullpath = os.path.join(data_path,f)
        image = glob.glob(os.path.join(fullpath,'*.png'))
       
        x_train,x_val = train_test_split(image,test_size = split_size)
        #print("folder name",f,len(x_train))
       
        for x in x_train:
         #   print(x)
            path_to_folder = os.path.join(train,f)
           
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)
            shutil.copy(x,path_to_folder)
       
        for y in x_val:
            path_to_folder = os.path.join(validation,f)
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)
            shutil.copy(y,path_to_folder)

In [ ]:
data_path = '../input/tt100k-traffic-sign/Train'
if not os.path.isdir('/kaggle/working/Training'):
    os.mkdir('/kaggle/working/Training')
if not os.path.isdir('/kaggle/working/Validation'):
    os.mkdir('/kaggle/working/Validation')

train = '/kaggle/working/Training'
validation = '/kaggle/working/Validation'
split(data_path,train=train,validation=validation,split_size = 0.1)

# Preparing Data Test 

In [ ]:
df =pd.read_csv("../input/tt100k-traffic-sign/Test.csv")
df['Path'] = df['Path'].str.replace('Test/','')
df.to_csv('/kaggle/working/Test1.csv')
os.mkdir("/kaggle/working/Test")

In [ ]:
def prepare_test(path_to_image,path_file):

    with open(path_file,"r") as csvfile:
        r= csv.reader(csvfile,delimiter =',')
  
        for i,row in enumerate(r):
            if i == 0: 
                continue
            label = row[-2]
            img_name = row[-1]
            
            dest = os.path.join('/kaggle/working/Test/',label)
            if not os.path.isdir(dest):
                os.makedirs(dest)
            
            to_move = os.path.join(path_to_image,img_name)
            shutil.copy(to_move,dest)

In [ ]:
import csv
path_to_image = "../input/tt100k-traffic-sign/Test"
path_file ='/kaggle/working/Test1.csv'
prepare_test(path_to_image,path_file)

# Begin create Model

In [ ]:
import tensorflow
from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, Flatten, BatchNormalization, GlobalAvgPool2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import Model
import PIL

# Create CNN

I have created 3 Convolutional layers with Maxpool layers in between. For all layers I have maintained same padding to ensure the layer's outputs to have same spatial dimensions as its inputs. This will ensure we dont miss out the info in the edges. These are all small images and we cant afford to miss out the edges

I also tried other activation functions. But only relu provides the highest accuracy


In [ ]:
def run_cnn(number_classes):
    
    i_shape = Input(shape=(60,60,3)) #Check the picture sizes randomly and decided. Can also done systematically by checking for all files and using median value
    x = Conv2D(32, (3,3), activation='relu', padding="same") (i_shape)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (3,3),  activation='relu', padding="same") (x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3,3),  activation='relu', padding="same") (x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)  
    
    x = Conv2D(128, (3,3),  activation='relu') (x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x) 
    
      
   # x = Flatten()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(1024,  activation='relu') (x)
    x = Dense(number_classes,activation="softmax")(x)
    

    return Model(inputs=i_shape,outputs=x)

With Elu activation function

In [ ]:
def run_cnn_elu(number_classes):
    
    i_shape = Input(shape=(60,60,3)) #Check the picture sizes randomly and decided. Can also done systematically by checking for all files and using median value
    x = Conv2D(32, (3,3), activation='elu', padding="same") (i_shape)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (3,3),  activation='elu', padding="same") (x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3,3),  activation='elu', padding="same") (x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)  
    
    x = Conv2D(128, (3,3),  activation='elu') (x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x) 
    
      
   # x = Flatten()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(1024,  activation='elu') (x)
    x = Dense(number_classes,activation="softmax")(x)
    

    return Model(inputs=i_shape,outputs=x)

In [ ]:
Data augmentation

In [ ]:
# Augmentation functions

# Horizontal Flip
def horizontal_flip(image):
    return cv2.flip(image, 1)

# Contrast Adjustment
def adjust_contrast(image, alpha=1.5):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=0)

# Add Gaussian Noise
def add_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

# Blurring (Gaussian Blur)
def blur_image(image, ksize=5):
    return cv2.GaussianBlur(image, (ksize, ksize), 0)

# Zoom
def zoom_image(image, zoom_factor=1.2):
    h, w = image.shape[:2]
    new_h, new_w = int(h / zoom_factor), int(w / zoom_factor)
    resized_image = cv2.resize(image, (new_w, new_h))
    pad_h = (h - new_h) // 2
    pad_w = (w - new_w) // 2
    padded_image = cv2.copyMakeBorder(resized_image, pad_h, pad_h, pad_w, pad_w, cv2.BORDER_REFLECT)
    return cv2.resize(padded_image, (w, h))

# Function to save augmented image
def save_image(image, output_dir, image_name, aug_type):
    output_path = os.path.join(output_dir, f"{image_name}_{aug_type}.jpg")
    cv2.imwrite(output_path, image)

# Directory paths
input_dir = '/input/tt100k-traffic-sign/'  # Set the path to your dataset
output_dir = '/kaggle/working/'    # Set the path to save augmented images

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through the dataset
for image_name in os.listdir(input_dir):
    image_path = os.path.join(input_dir, image_name)

    if image_name.endswith('.jpg') or image_name.endswith('.png'):
        # Read the image
        image = cv2.imread(image_path)

        # Apply Augmentations
        flipped_image = horizontal_flip(image)
        contrast_image = adjust_contrast(image)
        noisy_image = add_noise(image)
        blurred_image = blur_image(image)
        zoomed_image = zoom_image(image)

        # Save the augmented images
        save_image(flipped_image, output_dir, image_name.split('.')[0], 'flip')
        save_image(contrast_image, output_dir, image_name.split('.')[0], 'contrast')
        save_image(noisy_image, output_dir, image_name.split('.')[0], 'noise')
        save_image(blurred_image, output_dir, image_name.split('.')[0], 'blur')
        save_image(zoomed_image, output_dir, image_name.split('.')[0], 'zoom')

print("Data augmentation completed and saved to", output_dir)

# Train, Validation & Test

In [ ]:
def create_generators(train_dir,batch_size,validation_dir,test_dir):
  #  train_datagen1 = ImageDataGenerator(
   #   rescale=1./255,
    #  rotation_range=40,
     # width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest')
     
    train_datagen = ImageDataGenerator(
      rescale=1./255)
    
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
     
    train_generator=train_datagen.flow_from_directory(
         train_dir,target_size=(60,60),batch_size = batch_size,
         color_mode = 'rgb',class_mode ="categorical",
         shuffle=True)
     
    validation_generator=validation_datagen.flow_from_directory(
         validation_dir,target_size=(60,60),batch_size = batch_size,
         color_mode = 'rgb',class_mode ="categorical",
         shuffle=True)
     
    test_generator=test_datagen.flow_from_directory(
         test_dir,target_size=(60,60),batch_size = batch_size,
         color_mode = 'rgb',class_mode ="categorical",
         shuffle=True) 
     
    return(train_generator,validation_generator,test_generator)

# Fit the model

In [ ]:
train_dir = train
validation_dir = validation
test_dir = '/kaggle/working/Test'
batch_size = 60
number_classes = 43 

os.mkdir('/kaggle/working/model1')
path_to_save_model = '/kaggle/working/model1'
chkpt_saver = ModelCheckpoint(
    path_to_save_model, monitor='val_accuracy',
    mode ='max',save_best_only= True,
    save_freq='epoch',verbose = 1
    )


 
train_generator,validation_generator,test_generator = create_generators(train_dir,batch_size,validation_dir,test_dir)  
model =   run_cnn(number_classes)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator, # 2000 images = batch_size * steps
      epochs=15,batch_size = batch_size,
      validation_data=validation_generator, callbacks=[chkpt_saver]
    ) 
  
     
model = tensorflow.keras.models.load_model(path_to_save_model)  


In [ ]:
train_dir = train
validation_dir = validation
test_dir = '/kaggle/working/Test'
batch_size = 60
number_classes = 43 

os.mkdir('/kaggle/working/model_n')
path_to_save_model = '/kaggle/working/model_n'
chkpt_saver = ModelCheckpoint(
    path_to_save_model, monitor='val_accuracy',
    mode ='max',save_best_only= True,
    save_freq='epoch',verbose = 1
    )


 
train_generator,validation_generator,test_generator = create_generators(train_dir,batch_size,validation_dir,test_dir)  
model_n =   run_cnn_elu(number_classes)
model_n.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
history = model_n.fit(train_generator, # 2000 images = batch_size * steps
      epochs=15,batch_size = batch_size,
      validation_data=validation_generator, callbacks=[chkpt_saver]
    ) 
  
     
model_n = tensorflow.keras.models.load_model(path_to_save_model)  

In [ ]:

print("validation set")
model.evaluate(validation_generator)
print("test set")
model.evaluate(test_generator)

 
    

In [ ]:

print("validation set")
model_n.evaluate(validation_generator)
print("test set")
model_n.evaluate(test_generator)

In [ ]:

train_1_fnames = os.listdir( '/kaggle/working/Training/1' )
train_2_fnames = os.listdir( '/kaggle/working/Training/2' )

print(train_1_fnames[:10])
print(train_2_fnames[:10])


# Visualizing Intermediate Representations
To get a feel for what kind of features our convnet has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the convnet.

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]

visualization_model = tensorflow.keras.models.Model(inputs = model.input, outputs = successive_outputs)

sign1_files = [os.path.join('/kaggle/working/Training/1', f) for f in train_1_fnames]
sign2_files = [os.path.join('/kaggle/working/Training/2', f) for f in train_2_fnames]

img_path = random.choice(sign1_files + sign2_files)
img = load_img(img_path, target_size=(60, 60))  # this is a PIL image

x   = img_to_array(img)                          
x   = x.reshape((1,) + x.shape)                  

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]


# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )


# Plot the training/validation accuracy 

In [ ]:

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
def grad_cam(input_model, img, layer_name):
    grad_model = Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.array([img]))
        loss = predictions[:, np.argmax(predictions[0])]
    
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    
    # Average pooling of the gradients
    weights = np.mean(grads, axis=(0, 1))
    
    cam = np.dot(output, weights)
    
    # Resize the CAM to the size of the image
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()  # Normalize to [0, 1]
    
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    overlay = heatmap + np.float32(img)
    overlay = overlay / np.max(overlay)
    
    return np.uint8(255 * overlay)


img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))  # Resizing to match input shape of the model

# Generate GRAD-CAM heatmap for the last convolutional layer
heatmap = grad_cam(model, img, layer_name='conv2d_93')  # Example layer name; change based on the model

# Display the heatmap
plt.imshow(cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB))
plt.title("GRAD-CAM")
plt.axis('off')
plt.show()


In [ ]:
image_size = (224, 224)  # Image size for the models
batch_size = 32

# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    dataset_dir, 
    target_size=image_size, 
    batch_size=batch_size, 
    class_mode='categorical', 
    shuffle=False)

# 1. Define Models

# GoogleNet (InceptionV3)
def create_googlenet(input_shape=(224, 224, 3), num_classes=43):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# ResNet-34 (using ResNet50 as a close alternative)
def create_resnet(input_shape=(224, 224, 3), num_classes=43):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# VGG-16
def create_vgg16(input_shape=(224, 224, 3), num_classes=43):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Simple AlexNet Definition
def create_alexnet(input_shape=(224, 224, 3), num_classes=43):
    model = tf.keras.Sequential([
        layers.Conv2D(96, kernel_size=11, strides=4, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=3, strides=2),
        layers.Conv2D(256, kernel_size=5, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=3, strides=2),
        layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=3, strides=2),
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Example for DarkNet-53 and MicronNet-BF could be implemented similarly.

# Choose the model to train (you can swap between models as needed)
model = create_resnet()  # You can swap with create_googlenet(), create_vgg16(), create_alexnet(), etc.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model or load pre-trained model
model.fit(train_generator, epochs=10)
# model.load_weights('your_pretrained_model.h5')

# 2. GRAD-CAM Function
def grad_cam(input_model, img_array, layer_name):
    grad_model = Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]
    
    heatmap = tf.reduce_mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap.numpy()

# 3. Apply GRAD-CAM to All Images in the Dataset
def apply_grad_cam_to_dataset(model, generator, output_dir, layer_name):
    for i, (img_batch, label_batch) in enumerate(generator):
        for j in range(img_batch.shape[0]):
            img_array = np.expand_dims(img_batch[j], axis=0)
            heatmap = grad_cam(model, img_array, layer_name=layer_name)
            
            # Resize heatmap to image size and overlay
            heatmap = cv2.resize(heatmap, (image_size[0], image_size[1]))
            heatmap = np.uint8(255 * heatmap)
            heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
            
            # Original image
            img = np.uint8(255 * img_batch[j])
            superimposed_img = heatmap * 0.4 + img

            # Save image
            image_filename = f"grad_cam_{i}_{j}.jpg"
            output_path = os.path.join(output_dir, image_filename)
            cv2.imwrite(output_path, superimposed_img)
        
        if i >= len(generator) - 1:  # End the loop when the dataset is done
            break

# Specify the last convolutional layer of the model you want to visualize
# For ResNet-34, use 'conv5_block3_out', for GoogleNet use the layer name 'conv2d_93' (change based on architecture)
apply_grad_cam_to_dataset(model, train_generator, output_dir, layer_name='conv5_block3_out')

print("GRAD-CAM visualizations generated and saved.")
